# Imports

In [2]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

# basics
import numpy as np
import pandas as pd
import requests as r
import re
import geopandas as gpd
import json

# viz
import matplotlib.pyplot as plt
import seaborn as sns

# utils
import os
import time
from tqdm import tqdm
from geopy.geocoders import Nominatim
import geopandas as gpd
import pgeocode
import urllib
from shapely.geometry import Point
from itertools import chain
from kedro.config import ConfigLoader
from ppm.utils.external_get_data import (
    find_regex_cep,
    search_cep,
    find_census_area_by_zip,
    get_connection,
    scrapping_zipimoveis,
    find_cbg
)
from ppm.utils.readers import (
    reader,
    unzip_file
)
from ppm.utils.feature_process import (
    replace_names,
    transform_float,
    convert_to_float,
    replace_na_with_mean,
    replace_na_with_mode
)

In [3]:
which_test = "oos" 

# Parameters

In [14]:
uf = 'PB'
path_root_oos = os.path.join(
    '..', 'oos', # data or oos
)
path_root_data = os.path.join(
    '..', 'data', # data or oos
)
path_raw = os.path.join(
    path_root_data, '01_raw'
)
path_intermediate = os.path.join(
    path_root_oos, '02_intermediate'
)
path_intermediate_data = os.path.join(
    path_root_data, '02_intermediate'
)
path_primary = os.path.join(
    path_root_oos, '03_primary'
)
root_brasilapi = 'https://brasilapi.com.br/api'
root_zap = 'https://www.zapimoveis.com.br/'
url_path_zapimoveis = os.path.join(
    root_zap, 'venda/imoveis/pb+joao-pessoa/?pagina={}'
)
path_census_data = os.path.join(
    path_raw, 'PB_20171016'
)
path_census_data_csv = os.path.join(
    path_census_data,
    f'{uf}',
    f'Base informaçoes setores2010 universo {uf}',
    'CSV'
)
path_columns_cat = os.path.join(
    path_intermediate, "categorical_dict"
)
path_columns_cat_data = os.path.join(
    path_intermediate_data, "categorical_dict"
)

file_path_data_merged = os.path.join(
    path_intermediate, 'scrapping_data_concat.csv'
)
file_path_data_shp = os.path.join(
    path_raw, 
    'PB_Setores_2021',
    'PB_Setores_2021.shp'
)
file_path_shp_processed = os.path.join(
    path_intermediate_data, "data_shp_processed"
)
file_path_census_processed = os.path.join(
    path_intermediate_data, "data_census_processed.csv"
)
file_path_processed = os.path.join(
    path_primary, "data_processed.csv"
)
file_path_processed_concat = os.path.join(
    path_primary, "data_processed_concat.csv"
)
file_path_data_input = os.path.join(
    path_primary, "data_input.csv"
)
file_path_categorical_replaces = os.path.join(
    path_columns_cat, "{}.json"
)

# Read datasets

In [ ]:
# unzip
if not os.path.exists(path_census_data):
    unzip_file(
        path_census_data+'.zip',
        path_census_data
    )

In [ ]:
file_path_census_data = {
    i.split(".")[0]: os.path.join(path_census_data_csv, i)for i in os.listdir(path_census_data_csv) 
}

In [ ]:
data_census_merged = pd.read_csv(file_path_census_processed)
data_shp = gpd.read_file(file_path_shp_processed)

# Scrapping

In [ ]:
n_pages_init = 61
n_pages_final = 121 # 101

In [ ]:
data_results_scrapping = {}
for i in tqdm(range(n_pages_init, n_pages_final + 1)):
    print(url_path_zapimoveis.format(i))
    data_results_scrapping[
        f'page_n{i}'
    ] = scrapping_zipimoveis(url_path_zapimoveis.format(i))
    time.sleep(10)

In [ ]:
content_final = {}
errors = []
for page, content in tqdm(data_results_scrapping.items()):
    data_tmp = {}
    for idx, i in enumerate(list(content.items())):
        data_tmp[f'n{idx}'] = {}
        data_tmp[f'n{idx}']['ID'] = i[0].replace('id-','')
        data_tmp[f'n{idx}']['price'] = transform_float(i[1]['price'][0])
        data_tmp[f'n{idx}']['local'] = i[1]['local']
        data_tmp[f'n{idx}']['highlight'] = i[1]['highlight']
        for f_name, f_value in i[1]['features'].items():
            data_tmp[f'n{idx}'][f_name] = replace_names(f_value)
        data_tmp[f'n{idx}'] = pd.DataFrame(data_tmp[f'n{idx}'])
    try:
        content_final[page] = pd.concat(list(data_tmp.values()),
                                        ignore_index = True)
    except Exception as e:
        errors.append((page, e))
content_final = pd.concat(list(content_final.values()),
                          ignore_index = True).replace('', np.nan)

In [ ]:
cols_selected = [
    'areas',
    'bedrooms',
    'parking-spaces',
    'bathrooms'
]
string_rows = content_final[cols_selected].apply(lambda x: x.str.contains('-', na = False))

for i in cols_selected:
    values_changed = content_final[
        string_rows[i]
    ][i].apply(
        lambda x: (int(x.split("-")[0])+int(x.split("-")[1]))/2
    )
    content_final.loc[values_changed.index, i] = values_changed.values

In [ ]:
ceps = []
for i in tqdm(content_final['local']):
    try:
        ceps.append( search_cep(i)[1] )
    except:
        ceps.append( np.nan )
content_final['cep'] = ceps
content_final['cep'] = content_final['cep'].fillna(np.nan)

# Pre-processing

In [ ]:
cols_to_drop = [
    "bathrooms"
]
which_rows_to_drop = [
    'areas', 
    'bedrooms',
    'parking-spaces',
    'highlight'
]
id_tag = [
    "ID"
]
content_final_process = content_final.copy()

In [ ]:
content_final_process.price = content_final_process.price.apply(convert_to_float)

In [ ]:
lats = []
longs = []
cbg = []
nomi = pgeocode.Nominatim('br')
for local in  tqdm(content_final_process.local):
    if local!=None:
        address = local + f', {uf}, Brasil'
        try:
                                                                                                                        
            geolocator = Nominatim(user_agent="geolocalização")
            location = geolocator.geocode(address)

            lats.append(float(location.latitude))
            longs.append(float(location.longitude))

        except:
            lats.append(np.nan)
            longs.append(np.nan)

    else:
        lats.append(np.nan)
        longs.append(np.nan)

In [ ]:
content_final_process['lats'] = lats
content_final_process['longs'] = longs

In [ ]:
# Criar os objetos de ponto com as latitudes e longitudes
pontos = [Point(xy) for xy in zip(content_final_process['longs'], content_final_process['lats'])]

In [ ]:
cbgs = find_cbg(data_shp, pontos)
content_final_process['cd_setor'] = cbgs

In [ ]:
# order columns
id_columns = [
    "ID",
    "cd_setor"
]
content_final_process = content_final_process[id_columns+[i for i in content_final_process.columns if i not in id_columns]]
content_processed = content_final_process.dropna(subset = ["cd_setor"]).reset_index(drop = True)

In [ ]:
content_processed.cd_setor = content_processed.cd_setor.astype(str)
data_census_merged.cd_setor = data_census_merged.cd_setor.astype(str)

## Save scrapping

In [ ]:
content_processed.to_csv(file_path_processed)

# Concat census

In [ ]:
data_merged = content_processed.merge(
    data_census_merged, 
    on = ["cd_setor"],
    how = "left"
)
data_merged = data_merged.merge(
    data_shp[["cd_setor","nm_sit"]],
    on = ["cd_setor"],
    how="left"
)
data_merged.insert(
    4,
    "bairro",
    data_merged.local.apply(
        lambda x: x.split(", ")[-1]
    )
)

In [ ]:
data_merged.columns = data_merged.columns.str.replace(" ","_")
columns_na = list(data_merged.isna().sum()[data_merged.isna().sum()!=0].index)

In [ ]:
data_bairros, errors = replace_na_with_mean(data_merged, columns_na, 'bairro')
data_merged_not_nan = pd.concat(list(data_bairros.values()), ignore_index = True)
data_merged_not_nan = data_merged_not_nan[
    ~data_merged_not_nan['ID'].duplicated()
].reset_index(drop = True)

In [ ]:
data_merged_not_nan.drop(
    data_merged_not_nan.columns[data_merged_not_nan.columns.str.contains("situacao")],
    axis = 1,
    inplace = True)

## Save data processed

In [ ]:
data_merged_not_nan.to_csv(file_path_processed_concat)

# Drop columns with problemns

In [107]:
cods_cols = list(data_merged_not_nan.columns[data_merged_not_nan.columns.str.contains("cod")])
other_cols = [
    "cep",
    "lats",
    "longs",
    "local",
    "nm_sit"
]
all_columns = cods_cols + other_cols
data_merged_not_nan.drop(all_columns,
                         axis = 1, 
                         inplace = True)

# Categorical columns process

In [108]:
if not os.path.exists(path_columns_cat):
    os.makedirs(path_columns_cat)

In [109]:
cat_cols_nome = [
    'nome_grande_regiao_basico_pb',
    'nome_da_uf__basico_pb',
    'nome_da_meso_basico_pb',
    'nome_da_micro_basico_pb',
    'nome_da_rm_basico_pb',
    'nome_do_municipio_basico_pb',
    'nome_do_distrito_basico_pb',
    'nome_do_subdistrito_basico_pb',
]
data_merged_not_nan.drop(cat_cols_nome, axis = 1, inplace = True)

In [110]:
cat_cols = [
        "bairro"
    ] + [
        "nome_do_bairro_basico_pb",
        "highlight"
    ]
if os.path.exists(path_columns_cat_data):
    cat_dict_all = {}
    for col in os.listdir(path_columns_cat_data):
        json_cat = json.load(open(os.path.join(path_columns_cat_data, col), 'r'))
        cat_dict_all[col.split('.')[0]] = json_cat
else:
    cat_dict_all = {}
    for col in cat_cols:
        cat_dict = {}
        for idx, i in enumerate(data_merged_not_nan[col].unique()):
            cat_dict[i] = idx
        cat_dict_all[col] = cat_dict

In [111]:
data_merged_not_nan, e3 = replace_na_with_mode(
    data_merged_not_nan,
    ['highlight'],
    'bairro')

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 42.64it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 68.10it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 98.96it/s]

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 111.30it/s]

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1075.74it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 73.60it/s]

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 116.33it/s]

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 973.38it/s]

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1424.70it/s]

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 238.77it/s]

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1040.77it/s]

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 112.41it/s]

100%|███████████

In [112]:
data_final = data_merged_not_nan.copy()
data_final = data_final.replace("X", np.nan)

In [113]:
cat_tmp = {}
for column_name, content_cat in cat_dict_all.items():
    if len(set(data_final[column_name].unique()) - set(cat_dict_all[column_name].keys())) != 0:
        cat_tmp[column_name] = {}
        for i in set(data_final[column_name].unique()) - set(cat_dict_all[column_name].keys()):
            cat_tmp[column_name][i] = -1
        data_final[column_name].replace(
            cat_dict_all[column_name]|cat_tmp[column_name], inplace = True
        )
    else:
        data_final[column_name].replace(cat_dict_all[column_name], inplace = True)

JOÃO PESSOA (demais setores)
nan
Mumbaba
Costa E Silva
nan


### Save categorical labels

In [119]:
for i_col_cat, i_content_cat in cat_dict_all.items():
    with open(file_path_categorical_replaces.format(i_col_cat), 'w') as file:
        json.dump(i_content_cat, file)

### Process column with problem

In [120]:
for pro_col in data_final.columns:
    try:
        data_final[pro_col] = data_final[pro_col].astype(float)
    except:
        data_final[pro_col] = data_final[pro_col].astype(str).str.replace(",",".").astype(float)

## Drop bathrooms

In [121]:
data_final.drop("bathrooms", axis = 1, inplace = True)

## Drop last nans

In [122]:
data_final = data_final.dropna().reset_index(drop = True)

# Save dataset final

In [102]:
data_final.to_csv(file_path_data_input)